In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, f1_score


In [2]:
# Constants
IMG_SIZE = 128
NUM_FRAMES = 30
CHANNELS = 3
NUM_CLASSES = 2
BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 0.000005

In [5]:
# Function to load and preprocess video using optical flow
def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0, max(total_frames-1, 1), NUM_FRAMES).astype(int)

    prev_gray = None
    for idx in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])

            # Normalize and reshape mag to match RGB shape
            mag = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            mag = np.expand_dims(mag, axis=-1)  # Convert to (H, W, 1)
            frame = np.concatenate((frame, mag), axis=-1)  # Now (H, W, 4)

        else:
            frame = np.dstack((frame, np.zeros((IMG_SIZE, IMG_SIZE, 1))))  # Add empty motion channel

        prev_gray = gray
        frames.append(frame)

    cap.release()

    while len(frames) < NUM_FRAMES:
        frames.append(frames[-1])  # Pad with last frame

    return np.array(frames, dtype=np.float32) / 255.0



In [7]:
# Load dataset
def load_dataset(data_dir):
    X, y = [], []
    labels = {'non shop lifters': 0, 'shop lifters': 1}

    for label in labels.keys():
        folder_path = os.path.join(data_dir, label)
        for file in os.listdir(folder_path):
            if file.endswith(".mp4"):
                video_path = os.path.join(folder_path, file)
                X.append(load_video(video_path))
                y.append(labels[label])

    return np.array(X), np.array(y)


In [9]:
# Load dataset
data_dir = r"C:\Users\Eman\Downloads\Shop DataSet\Shop DataSet"
X, y = load_dataset(data_dir)

# Shuffle dataset
dataset = tf.data.Dataset.from_tensor_slices((X, y)).shuffle(len(X), reshuffle_each_iteration=True)

# Split dataset (80% train, 20% test)
train_size = int(0.8 * len(X))
train_dataset = dataset.take(train_size).batch(BATCH_SIZE)
test_dataset = dataset.skip(train_size).batch(BATCH_SIZE)

# Handle class imbalance
class_weights = compute_class_weight("balanced", classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in range(NUM_CLASSES)}


In [11]:
# Build Hybrid 3D CNN + LSTM model
def build_model():
    model = Sequential([
        Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(NUM_FRAMES, IMG_SIZE, IMG_SIZE, CHANNELS + 1)),
        MaxPooling3D(pool_size=(2, 2, 2)),
        BatchNormalization(),

        Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        BatchNormalization(),

        Conv3D(128, kernel_size=(3, 3, 3), activation='relu'),
        MaxPooling3D(pool_size=(2, 2, 2)),
        BatchNormalization(),

        TimeDistributed(Flatten()),  # Preserve temporal features
        LSTM(64, return_sequences=False),  # Temporal learning

        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [13]:
# Train the model
model = build_model()
model.summary()
model.fit(train_dataset, epochs=EPOCHS, validation_data=test_dataset, class_weight=class_weight_dict)


C:\Users\Eman\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                      │ (None, 28, 126, 126, 32)    │           3,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 14, 63, 63, 32)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 14, 63, 63, 32)      │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 12, 61, 61, 64)      │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 6, 30, 30, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 6, 30, 30, 64)       │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 4, 28, 28, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_2 (MaxPooling3D)       │ (None, 2, 14, 14, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 2, 14, 14, 128)      │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 2, 25088)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │       6,439,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,737,378 (25.70 MB)

 Trainable params: 6,736,930 (25.70 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.6305 - loss: 0.6595 - val_accuracy: 0.5246 - val_loss: 0.6916
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.8371 - loss: 0.4987 - val_accuracy: 0.4836 - val_loss: 0.7075
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.9111 - loss: 0.3498 - val_accuracy: 0.4754 - val_loss: 0.7064
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.9219 - loss: 0.2792 - val_accuracy: 0.5492 - val_loss: 0.6686
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.9417 - loss: 0.2106 - val_accuracy: 0.4590 - val_loss: 0.6599
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.9735 - loss: 0.1581 - val_accuracy: 0.7377 - val_loss: 0.5369
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.9747 - loss: 0.1538 - val_accuracy: 0.9344 - val_loss: 0.3978
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.9703 - loss: 0.1313 - val_accuracy: 0.9918 - val_loss:

In [15]:
# Save the model
model.save("shoplifting_3dcnn_lstm.h5")


In [17]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 312ms/step - accuracy: 0.9750 - loss: 0.0930
Test Loss: 0.1024
Test Accuracy: 0.9672


In [19]:
# Get true labels and predictions
y_true = np.concatenate([y for _, y in test_dataset], axis=0)
y_pred = np.argmax(model.predict(test_dataset), axis=1)

# Compute recall and F1-score
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Non-Shoplifting', 'Shoplifting']))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 331ms/step
Classification Report:
                 precision    recall  f1-score   support

Non-Shoplifting       0.52      0.53      0.52        64
    Shoplifting       0.46      0.45      0.46        58

       accuracy                           0.49       122
      macro avg       0.49      0.49      0.49       122
   weighted avg       0.49      0.49      0.49       122

Confusion Matrix:
[[34 30]
 [32 26]]
Recall: 0.4918
F1-score: 0.4913


In [33]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("shoplifting_3dcnn_lstm.h5")

# Constants
IMG_SIZE = 128
NUM_FRAMES = 30
CHANNELS = 3

# Function to preprocess a new video (same as training)
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0, max(total_frames-1, 1), NUM_FRAMES).astype(int)

    prev_gray = None
    for idx in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            mag = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            mag = np.expand_dims(mag, axis=-1)  
            frame = np.concatenate((frame, mag), axis=-1)  
        else:
            frame = np.dstack((frame, np.zeros((IMG_SIZE, IMG_SIZE, 1)))) 

        prev_gray = gray
        frames.append(frame)

    cap.release()

    while len(frames) < NUM_FRAMES:
        frames.append(frames[-1])  

    return np.array(frames, dtype=np.float32) / 255.0  

# Path to the test video
test_video_path = r"C:\Users\Eman\Desktop\task_6\Shop DataSet\shop lifters\videppppsss_30.mp4"

# Preprocess the test video
test_video = preprocess_video(test_video_path)

# Reshape for model input
test_video = np.expand_dims(test_video, axis=0)  # Shape: (1, NUM_FRAMES, IMG_SIZE, IMG_SIZE, 4)

# Make prediction
prediction = model.predict(test_video)
predicted_class = np.argmax(prediction)

# Define class labels
class_labels = ["Non-Shoplifting", "Shoplifting"]

# Print the result
print(f"Predicted Class: {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step
Predicted Class: Shoplifting


In [39]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("shoplifting_3dcnn_lstm.h5")

# Constants
IMG_SIZE = 128
NUM_FRAMES = 30
CHANNELS = 3

# Function to preprocess a new video (same as training)
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0, max(total_frames-1, 1), NUM_FRAMES).astype(int)

    prev_gray = None
    for idx in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            mag = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            mag = np.expand_dims(mag, axis=-1)  
            frame = np.concatenate((frame, mag), axis=-1)  
        else:
            frame = np.dstack((frame, np.zeros((IMG_SIZE, IMG_SIZE, 1)))) 

        prev_gray = gray
        frames.append(frame)

    cap.release()

    while len(frames) < NUM_FRAMES:
        frames.append(frames[-1])  

    return np.array(frames, dtype=np.float32) / 255.0  

# Path to the test video
test_video_path = r"C:\Users\Eman\Desktop\task_6\Shop DataSet\non shop lifters\shop_lifter_n_217.mp4"

# Preprocess the test video
test_video = preprocess_video(test_video_path)

# Reshape for model input
test_video = np.expand_dims(test_video, axis=0)  # Shape: (1, NUM_FRAMES, IMG_SIZE, IMG_SIZE, 4)

# Make prediction
prediction = model.predict(test_video)
predicted_class = np.argmax(prediction)

# Define class labels
class_labels = ["Non-Shoplifting", "Shoplifting"]

# Print the result
print(f"Predicted Class: {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step
Predicted Class: Non-Shoplifting


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("shoplifting_3dcnn_lstm.h5")

# Constants
IMG_SIZE = 128
NUM_FRAMES = 30
CHANNELS = 3

# Function to preprocess a new video (same as training)
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0, max(total_frames-1, 1), NUM_FRAMES).astype(int)

    prev_gray = None
    for idx in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            mag = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            mag = np.expand_dims(mag, axis=-1)  
            frame = np.concatenate((frame, mag), axis=-1)  
        else:
            frame = np.dstack((frame, np.zeros((IMG_SIZE, IMG_SIZE, 1)))) 

        prev_gray = gray
        frames.append(frame)

    cap.release()

    while len(frames) < NUM_FRAMES:
        frames.append(frames[-1])  

    return np.array(frames, dtype=np.float32) / 255.0  

# Path to the test video
test_video_path = r"C:\Users\Eman\Desktop\task_6\Shop DataSet\shop lifters\shop_lifter_3.mp4"

# Preprocess the test video
test_video = preprocess_video(test_video_path)

# Reshape for model input
test_video = np.expand_dims(test_video, axis=0)  # Shape: (1, NUM_FRAMES, IMG_SIZE, IMG_SIZE, 4)

# Make prediction
prediction = model.predict(test_video)
predicted_class = np.argmax(prediction)

# Define class labels
class_labels = ["Non-Shoplifting", "Shoplifting"]

# Print the result
print(f"Predicted Class: {class_labels[predicted_class]}")
